In [1]:
pip install transformers datasets torch scikit-learn pandas


Note: you may need to restart the kernel to use updated packages.


# Download dataset

In [2]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("abhi8923shriv/sentiment-analysis-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/sentiment-analysis-dataset


In [3]:
import pandas as pd

csv_path = path+"/train.csv"
data = pd.read_csv(csv_path, encoding='latin1')
print("\nSố dòng trong mỗi cột:")
print(len(data[['text', 'sentiment']]))

print("\nSố dòng thiếu trong mỗi cột:")
print(data[['text', 'sentiment']].isna().sum())

data_cleaned = data.dropna(subset=['text', 'sentiment'])

data_cleaned = data_cleaned[data_cleaned['text'].str.strip() != ""]
data_cleaned = data_cleaned[data_cleaned['sentiment'].str.strip() != ""]

data_cleaned.to_csv("cleaned_train.csv", index=False)

#================================================

csv_path = path+"/test.csv"
data = pd.read_csv(csv_path, encoding='latin1')

print("\nSố dòng thiếu trong mỗi cột:")
print(data[['text', 'sentiment']].isna().sum())

data_cleaned = data.dropna(subset=['text', 'sentiment'])

data_cleaned = data_cleaned[data_cleaned['text'].str.strip() != ""]
data_cleaned = data_cleaned[data_cleaned['sentiment'].str.strip() != ""]

data_cleaned.to_csv("cleaned_test.csv", index=False)


Số dòng trong mỗi cột:
27481

Số dòng thiếu trong mỗi cột:
text         1
sentiment    0
dtype: int64

Số dòng thiếu trong mỗi cột:
text         1281
sentiment    1281
dtype: int64


In [4]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv('/kaggle/working/cleaned_train.csv', encoding='latin-1')
tdf = pd.read_csv('/kaggle/working/cleaned_test.csv', encoding='latin-1')

print(df.head())

print(df.columns)

df = df[['text', 'sentiment']]
tdf = tdf[['text', 'sentiment']]

def convert_sentiment_label(x):
    if x == 'positive':
        return 1
    elif x == 'negative':
        return 0
    else:
        return 2

df['sentiment'] = df['sentiment'].apply(convert_sentiment_label)
tdf['sentiment'] = tdf['sentiment'].apply(convert_sentiment_label)

df['text'] = df['text'].astype(str)
tdf['text'] = tdf['text'].astype(str)

train_texts = df['text']
train_labels = df['sentiment']
val_texts= tdf['text']
val_labels = tdf['sentiment']

print("Train size:", len(train_texts))
print("Test size:", len(val_texts))

       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment Time of Tweet Age of User  \
0  I`d have responded, if I were going   neutral       morning        0-20   
1                             Sooo SAD  negative          noon       21-30   
2                          bullying me  negative         night       31-45   
3                       leave me alone  negative       morning       46-60   
4                        Sons of ****,  negative          noon       60-70   

       Country  Population -2020  Land Area (KmÂ²)  Density (P/KmÂ²)  
0  Afghanistan          38928346          652860.0 

In [5]:
from transformers import BertTokenizer, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
print("Tokenizer loaded successfully!")

def tokenize_function(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=220, return_tensors="pt")

train_encodings = tokenize_function(list(train_texts))
val_encodings = tokenize_function(list(val_texts))


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Tokenizer loaded successfully!


In [6]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)
print('run complete')


run complete


In [7]:
!pip install --upgrade peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 14.3 MB/s eta 0:00:00


In [8]:
from peft import PrefixEncoder, PrefixTuningConfig, get_peft_model
from transformers import BertForSequenceClassification
import torch
from torch import nn

config = PrefixTuningConfig(
    peft_type="PREFIX_TUNING",
    task_type="SEQ_CLS",
    num_virtual_tokens=50,
    token_dim=1024,
    num_transformer_submodules=1,
    num_attention_heads=16,
    encoder_hidden_size=1024
)

In [9]:
from torch.nn import CrossEntropyLoss
from torch import cuda
from torch.utils.data import DataLoader
from transformers import AdamW
from tqdm import tqdm
import time
import random
from torch.utils.data import Subset
from sklearn.metrics import accuracy_score
from transformers import BertForSequenceClassification, DistilBertForSequenceClassification
from collections import Counter

device = 'cuda' if cuda.is_available() else 'cpu'
data_len_list = [100, 200, 300, 400, 500, 1000]
epochs = 10
batch_size = 16
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

def EvalModel(model):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Validation Trained Model Accuracy: {accuracy:.4f}")
    print("===========================================")

# Training for many data_length
for data_length in data_len_list:
    model = BertForSequenceClassification.from_pretrained(
            "bert-large-uncased", num_labels=3)
    model.to(device)
    model.train()
    print("Training Data Length: ", data_length)

    optimizer = AdamW(model.parameters(), lr=5e-4)
    #random_indices = random.sample(range(len(train_dataset)), data_length)
    subset_indices = list(range(data_length))
    train_subset = Subset(train_dataset, subset_indices)
    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    
    label_counts = Counter()
    for batch in train_loader:
        labels = batch['labels']
        label_counts.update(labels.tolist())
    for label, count in label_counts.items():
        print(f"Label {label}: {count} samples")

    for epoch in range(epochs):
        start_time = time.time()
        total_loss = 0
        correct_predictions = 0
        total_samples = 0
        for batch in train_loader:
            optimizer.zero_grad()
            batch = {k: v.to(device) for k, v in batch.items()}
            labels = batch["labels"]
            outputs = model(**batch)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            correct_predictions += torch.sum(preds == labels).item()
            total_samples += labels.size(0)
        epoch_time = time.time() - start_time
        avg_loss = total_loss / len(train_loader)
        accuracy = correct_predictions / total_samples * 100
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}, "
              f"Accuracy: {correct_predictions / total_samples:.4f}, "
              f"Time: {epoch_time:.2f} seconds")

    # Eval model =====================================================
    EvalModel(model)






model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training Data Length:  100
Label 2: 43 samples
Label 0: 33 samples
Label 1: 24 samples


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 1.2220860719680786, Accuracy: 0.3800, Time: 7.44 seconds
Epoch 2/10, Loss: 1.1322765265192305, Accuracy: 0.4000, Time: 6.77 seconds
Epoch 3/10, Loss: 1.154542360986982, Accuracy: 0.3600, Time: 6.78 seconds
Epoch 4/10, Loss: 1.1888200896126884, Accuracy: 0.3300, Time: 6.78 seconds
Epoch 5/10, Loss: 1.0860763788223267, Accuracy: 0.4700, Time: 6.78 seconds
Epoch 6/10, Loss: 1.1381023951939173, Accuracy: 0.4000, Time: 6.80 seconds
Epoch 7/10, Loss: 1.1397927829197474, Accuracy: 0.4300, Time: 6.79 seconds
Epoch 8/10, Loss: 1.0972651072910853, Accuracy: 0.3500, Time: 6.80 seconds
Epoch 9/10, Loss: 1.221918991633824, Accuracy: 0.3200, Time: 6.82 seconds
Epoch 10/10, Loss: 1.166099292891366, Accuracy: 0.3100, Time: 6.83 seconds
Validation Trained Model Accuracy: 0.4046


...

Epoch 1/10, Loss: 1.1816784050315619, Accuracy: 0.3400, Time: 34.17 seconds
Epoch 2/10, Loss: 1.1359623149037361, Accuracy: 0.3400, Time: 34.18 seconds
Epoch 3/10, Loss: 1.178428702056408, Accuracy: 0.3440, Time: 34.19 seconds
Epoch 4/10, Loss: 1.129646709188819, Accuracy: 0.3660, Time: 34.17 seconds
Epoch 5/10, Loss: 1.1565310955047607, Accuracy: 0.3660, Time: 34.16 seconds
Epoch 6/10, Loss: 1.1499235536903143, Accuracy: 0.3420, Time: 34.20 seconds
Epoch 7/10, Loss: 1.1133504770696163, Accuracy: 0.3680, Time: 34.17 seconds
Epoch 8/10, Loss: 1.1391855869442225, Accuracy: 0.3680, Time: 34.17 seconds
Epoch 9/10, Loss: 1.1280756201595068, Accuracy: 0.3400, Time: 34.18 seconds
Epoch 10/10, Loss: 1.1506738606840372, Accuracy: 0.3660, Time: 34.17 seconds
Validation Trained Model Accuracy: 0.2832


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training Data Length:  1000
Label 0: 295 samples
Label 2: 390 samples
Label 1: 315 samples


/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 1.152097881786407, Accuracy: 0.3600, Time: 68.27 seconds
Epoch 2/10, Loss: 1.132719001126668, Accuracy: 0.3610, Time: 68.27 seconds
Epoch 3/10, Loss: 1.1655464683260237, Accuracy: 0.3270, Time: 68.25 seconds
Epoch 4/10, Loss: 1.1443409512913416, Accuracy: 0.3620, Time: 68.32 seconds
Epoch 5/10, Loss: 1.174426539549752, Accuracy: 0.3420, Time: 68.27 seconds
Epoch 6/10, Loss: 1.127492454316881, Accuracy: 0.3680, Time: 68.27 seconds
Epoch 7/10, Loss: 1.138550779176137, Accuracy: 0.3360, Time: 68.28 seconds
Epoch 8/10, Loss: 1.1333500742912292, Accuracy: 0.3510, Time: 68.30 seconds
Epoch 9/10, Loss: 1.1542118589083354, Accuracy: 0.3630, Time: 68.31 seconds
Epoch 10/10, Loss: 1.1266653216074383, Accuracy: 0.3600, Time: 68.32 seconds
Validation Trained Model Accuracy: 0.4046
